# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator
import random

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.5968677674749565),
 (1, 2.5968677674749565),
 (2, 2.5968677674749565),
 (3, 2.5968677674749565),
 (4, 2.5968677674749565)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18), ('what', 10)]),
 (1, [('is', 18)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.061091348855641825),
   (None, 0.06848470636730053),
   (None, 0.18042600062795322),
   (None, 0.19613450013704403),
   (None, 0.20582533893157973),
   (None, 0.25682452530643785),
   (None, 0.29631208319799296),
   (None, 0.2986384156149414),
   (None, 0.31221551069163833),
   (None, 0.32503506730767395),
   (None, 0.3400461704471651),
   (None, 0.3578724903843601),
   (None, 0.36735625115250825),
   (None, 0.4096399803678885),
   (None, 0.4388442178936276),
   (None, 0.4516136518150231),
   (None, 0.46111518878662794)]),
 (1,
  [(None, 0.6195560314991905),
   (None, 0.6309075254880534),
   (None, 0.6421632139869571),
   (None, 0.6860336302843129),
   (None, 0.690132207109684),
   (None, 0.7259351495036185),
   (None, 0.7783544505141263),
   (None, 0.8210820063551688),
   (None, 0.8233250593109074),
   (None, 0.8683561839456668),
   (None, 0.9555279430838479),
   (None, 0.9799903621625418),
   (None, 0.9995924461360887)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
import random

def MAP(num_list):
    return max(num_list)


def REDUCE(num_list):
    return max(num_list)


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[52, 32, 10, 25, 98, 48, 43, 84, 29, 44, 90, 64, 81, 92, 73, 22, 40, 20, 41, 60, 94, 39, 30, 56, 83, 27, 68, 13, 55, 93, 43, 49, 84, 49, 2, 14, 96, 33, 0, 30, 3, 23, 36, 12, 30, 39, 89, 63, 90, 46, 75, 93, 55, 98, 84, 76, 60, 94, 59, 51, 31, 91, 60, 43, 56, 84, 60, 21, 87, 39, 75, 88, 80, 71, 73, 12, 76, 73, 31, 65, 83, 86, 36, 3, 25, 76, 60, 91, 28, 6, 90, 67, 98, 91, 3, 96, 96, 86, 60, 10]
[[52, 32, 10, 25, 98], [48, 43, 84, 29, 44], [90, 64, 81, 92, 73], [22, 40, 20, 41, 60], [94, 39, 30, 56, 83], [27, 68, 13, 55, 93], [43, 49, 84, 49, 2], [14, 96, 33, 0, 30], [3, 23, 36, 12, 30], [39, 89, 63, 90, 46], [75, 93, 55, 98, 84], [76, 60, 94, 59, 51], [31, 91, 60, 43, 56], [84, 60, 21, 87, 39], [75, 88, 80, 71, 73], [12, 76, 73, 31, 65], [83, 86, 36, 3, 25], [76, 60, 91, 28, 6], [90, 67, 98, 91, 3], [96, 96, 86, 60, 10]]
98


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]


map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 10), (1, 98), (1, 26), (1, 16), (1, 21), (1, 0), (1, 60), (1, 49), (1, 92), (1, 85), (1, 9), (1, 94), (1, 8), (1, 80), (1, 40), (1, 64), (1, 93), (1, 53), (1, 16), (1, 61), (1, 78), (1, 79), (1, 42), (1, 61), (1, 9), (1, 43), (1, 55), (1, 92), (1, 71), (1, 99), (1, 91), (1, 92), (1, 67), (1, 64), (1, 77), (1, 52), (1, 93), (1, 35), (1, 77), (1, 47), (1, 58), (1, 26), (1, 27), (1, 48), (1, 61), (1, 5), (1, 81), (1, 7), (1, 96), (1, 33), (1, 35), (1, 1), (1, 83), (1, 53), (1, 8), (1, 38), (1, 47), (1, 98), (1, 74), (1, 17), (1, 22), (1, 20), (1, 40), (1, 84), (1, 43), (1, 77), (1, 44), (1, 64), (1, 32), (1, 83), (1, 38), (1, 37), (1, 46), (1, 68), (1, 68), (1, 78), (1, 45), (1, 2), (1, 80), (1, 24), (1, 12), (1, 80), (1, 48), (1, 69), (1, 40), (1, 95), (1, 45), (1, 32), (1, 29), (1, 65), (1, 62), (1, 23), (1, 40), (1, 63), (1, 89), (1, 67), (1, 9), (1, 0), (1, 60), (1, 92)]
[(1, [10, 98, 26, 16, 21, 0, 60, 49, 92, 85, 9, 94, 8, 80, 40, 64, 93, 53, 16, 61, 78, 79, 42, 61, 9, 43, 55, 

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
def group_by_key(iterable):
    sorted_data = sorted(iterable, key=lambda x: x[0])
    grouped_data = {}
    for key, value in sorted_data:
        if key not in grouped_data:
            grouped_data[key] = [value]
        else:
            grouped_data[key].append(value)
    result = []
    for key, value in grouped_data.items():
        result.append((key, value))
    return result

def MAP(num):
    return (1, num)


def REDUCE(key, nums: Iterator[NamedTuple]):
    sum = 0
    count = 0
    for n in nums:
        sum += n
        count += 1
    yield sum / count


def RECORDREADER(count):
    return [random.randint(0, 100) for i in range(count)]

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(flatten(map(lambda x: REDUCE(*x), shuffle_output)))
print(reduce_output)

[(1, 6), (1, 16), (1, 53), (1, 3), (1, 8), (1, 92), (1, 0), (1, 74), (1, 57), (1, 75), (1, 25), (1, 8), (1, 61), (1, 95), (1, 98), (1, 17), (1, 58), (1, 65), (1, 2), (1, 77), (1, 68), (1, 92), (1, 49), (1, 72), (1, 78), (1, 96), (1, 65), (1, 18), (1, 78), (1, 85), (1, 3), (1, 96), (1, 42), (1, 30), (1, 47), (1, 24), (1, 64), (1, 71), (1, 39), (1, 21), (1, 15), (1, 84), (1, 41), (1, 58), (1, 61), (1, 53), (1, 6), (1, 42), (1, 58), (1, 10), (1, 46), (1, 37), (1, 18), (1, 81), (1, 70), (1, 81), (1, 77), (1, 8), (1, 60), (1, 83), (1, 16), (1, 63), (1, 9), (1, 85), (1, 81), (1, 54), (1, 76), (1, 15), (1, 58), (1, 85), (1, 90), (1, 100), (1, 47), (1, 60), (1, 33), (1, 51), (1, 44), (1, 38), (1, 93), (1, 37), (1, 23), (1, 1), (1, 41), (1, 85), (1, 41), (1, 92), (1, 41), (1, 50), (1, 6), (1, 44), (1, 14), (1, 12), (1, 32), (1, 55), (1, 64), (1, 23), (1, 45), (1, 28), (1, 49), (1, 77)]
[(1, [6, 16, 53, 3, 8, 92, 0, 74, 57, 75, 25, 8, 61, 95, 98, 17, 58, 65, 2, 77, 68, 92, 49, 72, 78, 96, 65, 18

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [39]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs


# ===================================================================================================================================================================================

from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, word)

def REDUCE(key:str, value:Iterator[str]):
  yield key

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, ['', 'a', 'banana', 'it', 'what']), (1, ['is'])]

#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
from collections import defaultdict

def MAP(el_list):
    mapped_result = defaultdict(list)
    for t in el_list:
        if C(t):  # Предикат C
            mapped_result[t].append(t)
    return mapped_result.items()


def REDUCE(mapped_items):
    reduced_result = []
    print(mapped_items)
    for values in mapped_items:
        for value in values:
            reduced_result.append(value[0])
    return reduced_result

def C(t):
    return t[0] % 2 == 0  # Выбираем только кортежи с четным первым элементом


def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

record = RECORDREADER(100)
print(record)

part_count = 5
record_partitional = [record[d:d + part_count] for d in range(0, len(record), part_count)]
print(record_partitional)

print(REDUCE(list(map(lambda x: MAP(x), record_partitional))))

[(36, 53, 58), (92, 60, 69), (21, 61, 0), (38, 62, 85), (72, 98, 60), (61, 49, 86), (53, 89, 14), (63, 21, 100), (47, 36, 54), (49, 38, 82), (100, 45, 24), (31, 96, 81), (7, 34, 71), (65, 64, 93), (86, 98, 57), (22, 59, 47), (1, 54, 55), (51, 4, 57), (96, 78, 93), (31, 96, 2), (21, 46, 44), (64, 6, 21), (56, 70, 75), (19, 13, 28), (9, 40, 32), (11, 11, 15), (23, 43, 10), (72, 78, 27), (94, 94, 37), (84, 50, 81), (11, 44, 16), (10, 77, 64), (51, 17, 56), (83, 13, 33), (73, 6, 50), (12, 15, 89), (14, 8, 4), (13, 3, 68), (48, 40, 34), (66, 46, 81), (74, 65, 6), (38, 50, 99), (16, 3, 87), (51, 94, 49), (78, 66, 28), (38, 85, 11), (9, 47, 76), (32, 17, 34), (73, 32, 7), (7, 29, 13), (80, 28, 9), (31, 10, 21), (6, 82, 78), (97, 7, 88), (7, 8, 57), (22, 1, 48), (59, 36, 78), (4, 2, 38), (39, 92, 48), (56, 89, 40), (100, 50, 50), (52, 53, 78), (45, 99, 71), (87, 18, 85), (46, 92, 17), (18, 17, 14), (5, 31, 45), (91, 21, 83), (47, 65, 51), (7, 83, 88), (66, 23, 72), (16, 31, 37), (8, 95, 82), (

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [26]:
S = set([1 ,12, 56])

def MAP(t):
  res_list = []
  for el in t:
    if el in S:
      res_list.append(el)
    res = tuple(res_list)
    return (res, res)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(28, 0, 11), (82, 15, 47), (9, 47, 9), (24, 81, 4), (98, 0, 97), (75, 97, 99), (69, 87, 46), (56, 3, 33), (24, 11, 69), (45, 10, 77), (37, 41, 75), (25, 63, 77), (28, 36, 64), (65, 99, 66), (19, 87, 53), (3, 73, 48), (24, 45, 85), (55, 12, 34), (11, 83, 75), (96, 23, 16), (10, 49, 20), (10, 42, 10), (33, 85, 98), (94, 14, 89), (29, 96, 65), (5, 11, 45), (34, 24, 71), (17, 97, 37), (100, 13, 50), (27, 89, 29), (57, 56, 53), (41, 37, 60), (41, 40, 38), (56, 65, 90), (18, 4, 44), (53, 70, 96), (51, 45, 90), (32, 22, 76), (68, 38, 14), (15, 59, 55), (37, 25, 5), (26, 41, 17), (75, 74, 68), (55, 8, 55), (74, 22, 1), (82, 8, 48), (35, 4, 51), (35, 35, 20), (57, 17, 67), (46, 2, 37), (47, 46, 83), (95, 61, 34), (28, 98, 36), (36, 87, 90), (95, 10, 48), (28, 60, 91), (45, 41, 76), (99, 52, 59), (26, 43, 92), (75, 90, 41), (71, 49, 11), (75, 71, 18), (86, 29, 65), (6, 85, 4), (44, 33, 29), (37, 12, 1), (3, 26, 69), (39, 30, 69), (53, 17, 52), (90, 16, 74), (17, 73, 72), (11, 67, 73), (65, 50, 

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [40]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100), random.randint(0, 100)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x)[0], shuffle_output))
print(reduce_output)

[(70, 39, 20), (57, 60, 7), (86, 24, 22), (11, 74, 30), (37, 93, 34), (1, 9, 83), (33, 79, 48), (43, 98, 23), (92, 92, 2), (40, 40, 4), (89, 28, 55), (9, 97, 12), (49, 43, 78), (62, 40, 76), (66, 27, 70), (100, 20, 89), (3, 64, 27), (31, 54, 79), (6, 83, 19), (49, 100, 91), (10, 14, 78), (100, 59, 24), (2, 68, 87), (30, 80, 87), (93, 93, 22), (28, 10, 33), (13, 77, 63), (9, 62, 50), (100, 90, 63), (6, 95, 65), (1, 79, 78), (25, 66, 5), (73, 22, 77), (60, 17, 27), (92, 95, 7), (9, 51, 53), (48, 41, 17), (89, 72, 76), (86, 18, 86), (12, 71, 15), (70, 16, 100), (100, 51, 73), (14, 76, 64), (52, 68, 95), (100, 94, 47), (72, 53, 71), (14, 47, 31), (50, 53, 35), (66, 4, 8), (7, 0, 82), (76, 38, 94), (98, 36, 42), (6, 39, 26), (42, 36, 61), (6, 11, 32), (70, 20, 28), (65, 17, 46), (68, 62, 32), (66, 1, 31), (84, 47, 96), (38, 41, 73), (48, 81, 16), (60, 99, 44), (6, 38, 81), (65, 58, 72), (28, 65, 79), (2, 20, 15), (43, 27, 3), (19, 44, 61), (96, 47, 39), (58, 43, 96), (29, 87, 80), (24, 59, 

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [28]:
def MAP(t):
  return (t, t)


def REDUCE(key, values: Iterator[NamedTuple]):
  if len(values) == 2:
    return (key, key)

def RECORDREADER(count):
    return [(random.randint(0, 3), random.randint(0, 3)) for i in range(count)]

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[(0, 2), (0, 2), (1, 0), (3, 3), (1, 2), (1, 2), (0, 1), (3, 2), (0, 1), (3, 1), (2, 3), (1, 2), (2, 0), (3, 3), (1, 2), (0, 3), (3, 2), (2, 0), (0, 0), (3, 0), (3, 2), (3, 3), (2, 0), (1, 1), (3, 3), (3, 2), (3, 2), (3, 0), (3, 2), (1, 2), (2, 2), (0, 1), (1, 3), (0, 2), (3, 2), (3, 3), (3, 3), (0, 0), (3, 3), (0, 2), (1, 1), (0, 3), (0, 3), (1, 2), (2, 3), (1, 1), (3, 3), (0, 3), (3, 2), (1, 3), (2, 2), (1, 3), (1, 1), (1, 1), (1, 0), (2, 2), (1, 2), (2, 2), (1, 2), (3, 1), (0, 3), (3, 1), (2, 3), (1, 2), (0, 0), (1, 1), (0, 2), (3, 1), (0, 0), (0, 1), (1, 0), (2, 0), (1, 0), (2, 1), (1, 0), (0, 3), (0, 0), (3, 1), (1, 2), (0, 0), (2, 3), (3, 1), (3, 3), (0, 2), (0, 0), (0, 2), (3, 0), (0, 1), (0, 1), (2, 2), (1, 2), (2, 3), (3, 1), (2, 1), (2, 2), (1, 2), (2, 2), (3, 3), (1, 0), (2, 2)]
[((1, 2), (1, 2)), ((1, 3), (1, 3)), ((3, 1), (3, 1)), ((1, 2), (1, 2)), ((2, 1), (2, 1)), ((1, 1), (1, 1)), ((0, 3), (0, 3)), ((3, 3), (3, 3)), ((1, 1), (1, 1)), ((1, 2), (1, 2)), ((1, 1), (1, 1)), 

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [29]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple(count):
  data = tuple([(random.randint(0, 3), random.randint(0, 3)) for i in range(count)])
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
    return [get_random_tuple(3) for i in range(count)]

def MAP(t: Tuple):
  return (t.data, t.rel_id)

def REDUCE(key, values: Iterator[NamedTuple]):
  if values == [rels[0]]: # если соответствующее значение является списком  [R] , создайте пару  (t,t)
    return (key, key)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print(reduce_output)

[<__main__.Tuple object at 0x7b5f86d34d30>, <__main__.Tuple object at 0x7b5f86d37010>, <__main__.Tuple object at 0x7b5f86d358a0>, <__main__.Tuple object at 0x7b5f86d36dd0>, <__main__.Tuple object at 0x7b5f86d37b50>, <__main__.Tuple object at 0x7b5f86d36440>, <__main__.Tuple object at 0x7b5f86d355d0>, <__main__.Tuple object at 0x7b5f86d360b0>, <__main__.Tuple object at 0x7b5f86d37a00>, <__main__.Tuple object at 0x7b5f86d37d00>, <__main__.Tuple object at 0x7b5f86d354e0>, <__main__.Tuple object at 0x7b5f86d37820>, <__main__.Tuple object at 0x7b5f84105990>, <__main__.Tuple object at 0x7b5f84107430>, <__main__.Tuple object at 0x7b5f84104be0>, <__main__.Tuple object at 0x7b5f841079a0>, <__main__.Tuple object at 0x7b5f84106e60>, <__main__.Tuple object at 0x7b5f841051b0>, <__main__.Tuple object at 0x7b5f841052d0>, <__main__.Tuple object at 0x7b5f84107820>, <__main__.Tuple object at 0x7b5f84105900>, <__main__.Tuple object at 0x7b5f84106650>, <__main__.Tuple object at 0x7b5f841078e0>, <__main__.

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [30]:
rels = [1, 2]
class Tuple: # класс кортежа, который хранит данные и имя представления из массива rels выше
  def __init__(self, data: tuple, rel_id: int):
    self.data = data
    self.rel_id = rel_id

def get_random_tuple():
  data = (random.randint(0, 3), random.randint(0, 3))
  rel_id = rels[random.randint(0, len(rels) - 1)]
  return Tuple(data, rel_id)


def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  if t.rel_id == rels[0]:
    return (t.data[1], (t.rel_id, t.data[0]))
  else:
    return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values: Iterator[NamedTuple]):
  res = []
  for v in values:
    res.append((v[0], key, v[1]))
  return res

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[<__main__.Tuple object at 0x7b5f86d6c8b0>, <__main__.Tuple object at 0x7b5f86d6c4f0>, <__main__.Tuple object at 0x7b5f86d6f130>, <__main__.Tuple object at 0x7b5f86d6fa00>, <__main__.Tuple object at 0x7b5f86d6d9c0>, <__main__.Tuple object at 0x7b5f86d6c7c0>, <__main__.Tuple object at 0x7b5f86d6d480>, <__main__.Tuple object at 0x7b5f86d6cbe0>, <__main__.Tuple object at 0x7b5f86d6fcd0>, <__main__.Tuple object at 0x7b5f86d6edd0>, <__main__.Tuple object at 0x7b5f86d6c3a0>, <__main__.Tuple object at 0x7b5f86d6ebf0>, <__main__.Tuple object at 0x7b5f86d6f2b0>, <__main__.Tuple object at 0x7b5f86d6e470>, <__main__.Tuple object at 0x7b5f84105330>, <__main__.Tuple object at 0x7b5f84106170>, <__main__.Tuple object at 0x7b5f84106fb0>, <__main__.Tuple object at 0x7b5f84106fe0>, <__main__.Tuple object at 0x7b5f84104280>, <__main__.Tuple object at 0x7b5f84105960>, <__main__.Tuple object at 0x7b5f84105270>, <__main__.Tuple object at 0x7b5f841052a0>, <__main__.Tuple object at 0x7b5f84105180>, <__main__.

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [31]:
def get_random_tuple():
  return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):
  return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
  return (t[0], t[1])

def tetta(values):
  return sum(values)


def REDUCE(key, values: Iterator[NamedTuple]):
  x = tetta(values)
  return (key, x)

def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(100)
print(record)
map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print(map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)

print(shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print(reduce_output)

[(3, 3, 2), (2, 2, 3), (3, 2, 3), (1, 1, 1), (0, 3, 0), (2, 0, 2), (3, 0, 3), (2, 0, 1), (0, 2, 1), (3, 3, 3), (2, 2, 1), (3, 3, 0), (1, 3, 2), (0, 1, 1), (0, 1, 1), (1, 2, 0), (2, 2, 0), (3, 3, 1), (0, 2, 0), (2, 1, 0), (3, 1, 0), (0, 0, 1), (0, 2, 1), (1, 3, 0), (3, 2, 1), (0, 3, 0), (0, 1, 2), (0, 0, 2), (1, 0, 0), (1, 2, 0), (1, 3, 1), (1, 2, 0), (1, 1, 2), (2, 1, 0), (1, 3, 2), (3, 2, 0), (0, 3, 0), (1, 0, 2), (3, 3, 0), (1, 0, 3), (3, 2, 2), (0, 0, 2), (2, 1, 1), (2, 3, 2), (2, 0, 2), (1, 1, 3), (2, 0, 1), (3, 3, 1), (2, 1, 3), (2, 2, 0), (2, 3, 3), (2, 2, 3), (2, 1, 2), (0, 0, 3), (1, 1, 3), (1, 3, 3), (3, 0, 1), (2, 0, 0), (2, 3, 1), (1, 3, 1), (3, 0, 3), (1, 1, 3), (2, 1, 1), (2, 3, 2), (1, 0, 3), (0, 0, 3), (0, 0, 2), (1, 2, 0), (3, 1, 3), (1, 1, 3), (2, 3, 2), (3, 3, 0), (3, 0, 2), (2, 0, 1), (1, 3, 2), (1, 1, 1), (0, 1, 3), (0, 0, 0), (0, 2, 0), (3, 0, 3), (3, 2, 2), (0, 2, 2), (1, 0, 3), (2, 2, 2), (3, 3, 0), (1, 0, 1), (0, 1, 1), (0, 3, 3), (1, 3, 3), (1, 1, 2), (2, 3, 0)

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [32]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [33]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j, k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [34]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [35]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [36]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i,j]) # первая матрица

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j,k]) # вторая матрица

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)


def REDUCE_MUL(key, values):
  res_el_value = 0
  for v in values:
    res_el_value += v
  yield (key, res_el_value)


def GET_JOINED():
  for j in joined:
    yield j

joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [37]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица
  yield second_mat


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.10518278979887362), ((0, 1), 0.09114144084317408), ((0, 2), 0.08145423423021143), ((0, 3), 0.0045014419123484695), ((0, 4), 0.055348185054376046), ((0, 5), 0.13986792866939882), ((0, 6), 0.11576087775665726), ((0, 7), 0.11316343937252667), ((0, 8), 0.052833597114590125), ((0, 9), 0.11797175950288269), ((0, 10), 0.14200634807434837), ((0, 11), 0.10147727910402378), ((0, 12), 0.12125350538035157), ((0, 13), 0.05024467634565205), ((0, 14), 0.0984963866039474), ((0, 15), 0.10553418073369976), ((0, 16), 0.02231126544160559), ((0, 17), 0.13247225693873163), ((0, 18), 0.01701473614641779), ((0, 19), 0.044384052194563414), ((0, 20), 0.13514606285766823), ((0, 21), 0.13868997677932657), ((0, 22), 0.0915169101293526), ((0, 23), 0.10194796007440556), ((0, 24), 0.09680569847131276), ((0, 25), 0.0735914714110834), ((0, 26), 0.015120210613279294), ((0, 27), 0.10973465215845869), ((0, 28), 0.09613605235333703), ((0, 29), 0.118303116604102

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [38]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

# ===================================================================================================================================================================================

import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j])) # первая матрица

  global maps
  split_size =  int(np.ceil(len(first_mat)/maps))
  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k])) # вторая матрица

  split_size =  int(np.ceil(len(second_mat)/maps))
  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]


def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))


def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]


def MAP_MUL(k1, v1):
  yield (k1, v1)


def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)
np.allclose(reference_solution, asmatrix(solution)) # should return true


126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.0010642562887405685), ((0, 1), 0.038416817772748364), ((0, 2), 0.00879184967213078), ((0, 3), 0.057579886778866204), ((0, 4), 0.0014064699196008575), ((0, 5), 0.0833157436539128), ((0, 6), 0.014627193804310748), ((0, 7), 0.0744152016201859), ((0, 8), 0.05724508367392686), ((0, 9), 0.04026453416552845), ((0, 10), 0.00011393645169451594), ((0, 11), 0.03327573865273854), ((0, 12), 0.09000721510394193), ((0, 13), 0.08517413385962819), ((0, 14), 0.01008180248166741), ((0, 15), 0.0364747383394248), ((0, 16), 0.01720071129551431), ((0, 17), 0.021950869072464376), ((0, 18), 0.035687547777396564), ((0, 19), 0.033197890631096925), ((0, 20), 0.0819250740571198), ((0, 21), 0.01750880609806308), ((0, 22), 0.06123435856050151), ((0, 23), 0.0327450655485359), ((0, 24), 0.04319361753659657), ((0, 25), 0.04883041486122532), ((0, 26), 0.004599291028257314), ((0, 27), 0.06607371628111379), ((0, 28), 0.07620910521588078), ((0, 29), 0.065154806

True